# Precalculate FLORIS Solutions

Use FLASC tools to precalculate the FLORIS solutions across wind speeds and directions without wake steering to enable faster comparisons in later steps. The FLORIS solutions based on four different wake models are then saved in the "/precalculated_floris_solutions" folder. 

In [4]:
from pathlib import Path
import numpy as np
from time import perf_counter as timerpc

from floris.tools import ParallelComputingInterface
from flasc import floris_tools as ftools

from flasc.utilities_examples import load_floris_smarteole as load_floris

In [5]:
# User settings
max_workers = 16
wake_models = ["jensen", "turbopark", "gch", "cc"]

In [6]:
# Precalculate FLORIS solutions
root_path = Path.cwd() / "precalculated_floris_solutions"
root_path.mkdir(exist_ok=True)

for wake_model in wake_models:
    fn = root_path / "df_fi_approx_{:s}.ftr".format(wake_model)
    if fn.is_file():
        print("FLORIS table for '{:s}' model exists. Skipping...".format(wake_model))
        continue

    start_time = timerpc()
    print("Precalculating FLORIS table for '{:s}' model...".format(wake_model))
    fi, _ = load_floris(wake_model=wake_model)
    fi_pci = ParallelComputingInterface(
        fi=fi,
        max_workers=max_workers,
        n_wind_direction_splits=max_workers,
        print_timings=True,
    )
    df_fi_approx = ftools.calc_floris_approx_table(
        fi=fi_pci,
        wd_array=np.arange(0.0, 360.01, 3.0),
        ws_array=np.arange(1.0, 30.01, 1.0),
        ti_array=[0.03, 0.06, 0.09, 0.12, 0.15],
    )
    end_time = timerpc()
    print("Computation time: {:.2f} s".format(end_time - start_time))
    df_fi_approx.to_feather(fn)

Precalculating FLORIS table for 'jensen' model...
Generating a df_approx table of FLORIS solutions covering a total of 18150 cases.
Total time spent for parallel calculation (16 workers): 1.292 s
  Time spent in parallel preprocessing: 0.004 s
  Time spent in parallel loop execution: 1.288 s.
  Time spent in parallel postprocessing: 0.001 s
Total time spent for parallel calculation (16 workers): 1.258 s
  Time spent in parallel preprocessing: 0.003 s
  Time spent in parallel loop execution: 1.254 s.
  Time spent in parallel postprocessing: 0.001 s
Total time spent for parallel calculation (16 workers): 1.276 s
  Time spent in parallel preprocessing: 0.003 s
  Time spent in parallel loop execution: 1.272 s.
  Time spent in parallel postprocessing: 0.001 s
Total time spent for parallel calculation (16 workers): 1.193 s
  Time spent in parallel preprocessing: 0.003 s
  Time spent in parallel loop execution: 1.189 s.
  Time spent in parallel postprocessing: 0.001 s
Total time spent for par

floris.simulation.wake_deflection.gauss.GaussVelocityDeflection WARNING WARNING: Deflection with the TurbOPark model has not been fully validated.This is an initial implementation, and we advise you use at your own riskand perform a thorough examination of the results.
floris.simulation.wake_deflection.gauss.GaussVelocityDeflection WARNING WARNING: Deflection with the TurbOPark model has not been fully validated.This is an initial implementation, and we advise you use at your own riskand perform a thorough examination of the results.
floris.simulation.wake_deflection.gauss.GaussVelocityDeflection WARNING WARNING: Deflection with the TurbOPark model has not been fully validated.This is an initial implementation, and we advise you use at your own riskand perform a thorough examination of the results.
floris.simulation.wake_deflection.gauss.GaussVelocityDeflection WARNING WARNING: Deflection with the TurbOPark model has not been fully validated.This is an initial implementation, and we ad

Total time spent for parallel calculation (16 workers): 1.649 s
  Time spent in parallel preprocessing: 0.003 s
  Time spent in parallel loop execution: 1.645 s.
  Time spent in parallel postprocessing: 0.001 s


floris.simulation.wake_deflection.gauss.GaussVelocityDeflection WARNING WARNING: Deflection with the TurbOPark model has not been fully validated.This is an initial implementation, and we advise you use at your own riskand perform a thorough examination of the results.
floris.simulation.wake_deflection.gauss.GaussVelocityDeflection WARNING WARNING: Deflection with the TurbOPark model has not been fully validated.This is an initial implementation, and we advise you use at your own riskand perform a thorough examination of the results.
floris.simulation.wake_deflection.gauss.GaussVelocityDeflection WARNING WARNING: Deflection with the TurbOPark model has not been fully validated.This is an initial implementation, and we advise you use at your own riskand perform a thorough examination of the results.
floris.simulation.wake_deflection.gauss.GaussVelocityDeflection WARNING WARNING: Deflection with the TurbOPark model has not been fully validated.This is an initial implementation, and we ad

Total time spent for parallel calculation (16 workers): 1.797 s
  Time spent in parallel preprocessing: 0.003 s
  Time spent in parallel loop execution: 1.793 s.
  Time spent in parallel postprocessing: 0.001 s


...

Total time spent for parallel calculation (16 workers): 1.906 s
  Time spent in parallel preprocessing: 0.003 s
  Time spent in parallel loop execution: 1.903 s.
  Time spent in parallel postprocessing: 0.001 s


floris.simulation.wake_deflection.gauss.GaussVelocityDeflection WARNING WARNING: Deflection with the TurbOPark model has not been fully validated.This is an initial implementation, and we advise you use at your own riskand perform a thorough examination of the results.
floris.simulation.wake_deflection.gauss.GaussVelocityDeflection WARNING WARNING: Deflection with the TurbOPark model has not been fully validated.This is an initial implementation, and we advise you use at your own riskand perform a thorough examination of the results.
floris.simulation.wake_deflection.gauss.GaussVelocityDeflection WARNING WARNING: Deflection with the TurbOPark model has not been fully validated.This is an initial implementation, and we advise you use at your own riskand perform a thorough examination of the results.
floris.simulation.wake_deflection.gauss.GaussVelocityDeflection WARNING WARNING: Deflection with the TurbOPark model has not been fully validated.This is an initial implementation, and we ad

Total time spent for parallel calculation (16 workers): 1.952 s
  Time spent in parallel preprocessing: 0.003 s
  Time spent in parallel loop execution: 1.948 s.
  Time spent in parallel postprocessing: 0.001 s


floris.simulation.wake_deflection.gauss.GaussVelocityDeflection WARNING WARNING: Deflection with the TurbOPark model has not been fully validated.This is an initial implementation, and we advise you use at your own riskand perform a thorough examination of the results.
floris.simulation.wake_deflection.gauss.GaussVelocityDeflection WARNING WARNING: Deflection with the TurbOPark model has not been fully validated.This is an initial implementation, and we advise you use at your own riskand perform a thorough examination of the results.
floris.simulation.wake_deflection.gauss.GaussVelocityDeflection WARNING WARNING: Deflection with the TurbOPark model has not been fully validated.This is an initial implementation, and we advise you use at your own riskand perform a thorough examination of the results.
floris.simulation.wake_deflection.gauss.GaussVelocityDeflection WARNING WARNING: Deflection with the TurbOPark model has not been fully validated.This is an initial implementation, and we ad

Total time spent for parallel calculation (16 workers): 2.059 s
  Time spent in parallel preprocessing: 0.003 s
  Time spent in parallel loop execution: 2.055 s.
  Time spent in parallel postprocessing: 0.001 s
Finished calculating the FLORIS solutions for the dataframe.
Computation time: 11.83 s
Precalculating FLORIS table for 'gch' model...
Generating a df_approx table of FLORIS solutions covering a total of 18150 cases.
Total time spent for parallel calculation (16 workers): 1.584 s
  Time spent in parallel preprocessing: 0.003 s
  Time spent in parallel loop execution: 1.580 s.
  Time spent in parallel postprocessing: 0.001 s
Total time spent for parallel calculation (16 workers): 1.631 s
  Time spent in parallel preprocessing: 0.003 s
  Time spent in parallel loop execution: 1.627 s.
  Time spent in parallel postprocessing: 0.001 s
Total time spent for parallel calculation (16 workers): 1.702 s
  Time spent in parallel preprocessing: 0.003 s
  Time spent in parallel loop execution